<a href="https://colab.research.google.com/github/sujithkanna2004/Bookchat/blob/main/bookchat_(t8)_end_goal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyPDF2 requests spacy pyttsx3 openrouter gradio speechrecognition gtts pydub langchain


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 55.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!apt-get install espeak


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  espeak-data libespeak1 libportaudio2 libsonic0
The following NEW packages will be installed:
  espeak espeak-data libespeak1 libportaudio2 libsonic0
0 upgraded, 5 newly installed, 0 to remove and 29 not upgraded.
Need to get 1,382 kB of archives.
After this operation, 3,178 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libsonic0 amd64 0.2.0-11build1 [10.3 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 espeak-data amd64 1.48.15+dfsg-3 [1,085 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libespeak1 amd64 1.48.15+dfsg-3 [156 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy/universe amd64 espeak amd64 1.48.15+dfsg-3 [64.2 kB]
Fetched 1,382 kB in 2s (611 kB

In [ ]:
import os

os.environ["OPENROUTER_API_KEY"] = "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"


In [ ]:
import requests

url = "https://openrouter.ai/api/v1/models"
headers = {"Authorization": f"Bearer {os.getenv('OPENROUTER_API_KEY')}"}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    print("✅ API Key is working!")
else:
    print("❌ API Key is incorrect. Check again.")



✅ API Key is working!


In [ ]:
%%writefile main.py
import re
import PyPDF2
import requests
import os
import spacy
from gtts import gTTS
import tempfile

# Load NLP model for character extraction
nlp = spacy.load("en_core_web_sm")

# OpenRouter API Key
OPENROUTER_API_KEY = "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"

# Dictionary to store character memory
character_memory = {}

def extract_text_from_pdf(pdf_file):
    """Extracts text from a PDF book file."""
    try:
        with open(pdf_file, "rb") as file:
            reader = PyPDF2.PdfReader(file)
            text = ""
            for page in reader.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text + "\n"
        return text
    except Exception as e:
        return f"⚠️ Error reading PDF: {str(e)}"

def extract_characters_and_author(book_text):
    """Extracts characters and author using NLP (Spacy)."""
    doc = nlp(book_text[:100000])  # Process only the first 100,000 characters
    characters = list(set(ent.text for ent in doc.ents if ent.label_ == "PERSON"))
    author_match = re.search(r"by\s+([\w\s]+)", " ".join(book_text.split()[:500]), re.IGNORECASE)
    author = author_match.group(1) if author_match else "Unknown Author"
    characters.append("Author")
    return author, characters

def process_uploaded_book(book_file):
    """Processes the uploaded book and extracts details."""
    book_text = extract_text_from_pdf(book_file)
    if "⚠️" in book_text:
        return book_text, []
    author, characters = extract_characters_and_author(book_text)
    return f"✍️ Author: {author}\n🎭 Characters: {', '.join(characters)}", characters

def generate_emotional_response(book_title, character, user_input, emotion):
    """Generates a response with deep emotional variation."""
    system_message = f"You are {character} from the book '{book_title}'. Stay fully in character."

    emotion_prompt = {
        "normal": "Respond normally but not as an AI, but as a living breathing character.",
        "sad": "Respond as if you're crying and feeling deep sadness. *sobs*... I... I can't... It's too much... 😭",
        "angry": "Respond with extreme rage, yelling and cursing. *SCREAMS* HOW DARE YOU?! 😡",
        "flirty": "Respond seductively. Ohh~ you really know how to get me going~ 😉 (soft chuckle)..."
    }

    # Maintain character memory
    memory = character_memory.get(character, "")

    prompt = f"""
    {system_message}
    {emotion_prompt.get(emotion, 'Respond normally.')}
    Remember past conversations: {memory}
    User: {user_input}
    """

    try:
        response = requests.post(
            "https://openrouter.ai/api/v1/chat/completions",
            headers={"Authorization": f"Bearer {OPENROUTER_API_KEY}"},
            json={
                "model": "openai/gpt-3.5-turbo",
                "messages": [
                    {"role": "system", "content": system_message},
                    {"role": "user", "content": prompt}
                ]
            }
        )
        response_data = response.json()
        bot_reply = response_data["choices"][0]["message"]["content"] if "choices" in response_data else "⚠️ No valid response received."

        # Store memory
        character_memory[character] = bot_reply

        return bot_reply
    except Exception as e:
        return f"⚠️ Chat error: {str(e)}"

def text_to_speech(response_text):
    """Converts text response into speech using gTTS and returns the audio file path."""
    tts = gTTS(response_text, lang="en")  # Convert text to speech
    temp_audio = tempfile.NamedTemporaryFile(delete=False, suffix=".mp3")  # Create temp file
    tts.save(temp_audio.name)  # Save audio file
    return temp_audio.name  # Return file path

def chat_with_character(book_title, character, user_input, available_characters, emotion):
    """Generates an emotional response, remembers chat history, and plays voice output."""
    if not book_title:
        return "⚠️ No book uploaded. Please upload a book first!", None

    if character not in available_characters:
        return f"⚠️ '{character}' is not in the story. Please choose from: {', '.join(available_characters)}", None

    response_text = generate_emotional_response(book_title, character, user_input, emotion)
    audio_path = text_to_speech(response_text)  # Convert text to speech

    return response_text, audio_path  # Return both text & audio file


Writing main.py


In [ ]:
%%writefile gui.py
import gradio as gr
import os
from main import process_uploaded_book, chat_with_character

uploaded_book = None
available_characters = []  # Store extracted characters

def upload_book(file):
    """Handles book upload and extracts characters & author."""
    global uploaded_book, available_characters
    uploaded_book = file.name  # Save file path

    # Extract characters & author
    book_info, characters = process_uploaded_book(uploaded_book)
    available_characters = characters  # Store characters globally

    return book_info, gr.update(choices=characters, value=characters[0] if characters else "")

def remove_book():
    """Removes the uploaded book."""
    global uploaded_book, available_characters
    uploaded_book = None
    available_characters = []
    return "📕 Book removed! Please upload a new one.", gr.update(choices=[], value="")

def chat(character, emotion, user_input):
    """Handles chat interaction with selected character and emotion."""
    if not uploaded_book:
        return "⚠️ Please upload a book first!", None

    response_text, audio_path = chat_with_character(uploaded_book, character, user_input, available_characters, emotion)
    return response_text, audio_path  # Return text & audio path

# Define Gradio interface
with gr.Blocks() as iface:
    gr.Markdown("# 📚 BookChat Chatbot")

    book_upload = gr.File(label="Upload your book (PDF or TXT)")
    upload_btn = gr.Button("📂 Upload Book")
    remove_btn = gr.Button("❌ Remove Book")
    book_info = gr.Textbox(label="Book Details", interactive=False)

    character_dropdown = gr.Dropdown(label="Select Character", choices=[])
    emotion_dropdown = gr.Dropdown(label="Select Emotion", choices=["Normal", "Sad", "Angry", "Flirty"], value="Normal")
    user_input = gr.Textbox(label="Your Message")
    chat_btn = gr.Button("💬 Chat")
    bot_response = gr.Textbox(label="Chatbot Response", interactive=False)
    bot_audio = gr.Audio(label="Character Voice", autoplay=True)  # Audio output

    upload_btn.click(upload_book, inputs=[book_upload], outputs=[book_info, character_dropdown])
    remove_btn.click(remove_book, outputs=[book_info, character_dropdown])
    chat_btn.click(chat, inputs=[character_dropdown, emotion_dropdown, user_input], outputs=[bot_response, bot_audio])

iface.launch(share=True)


Writing gui.py


In [ ]:
!python gui.py



* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://e6177267e615914019.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
Keyboard interruption in main thread... closing server.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 2960, in block_thread
    time.sleep(0.1)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/content/gui.py", line 54, in <module>
    iface.launch(share=True)
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 2866, in launch
    self.block_thread()
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 2964, in block_thread
    self.server.close()
  File "/usr/local/lib/python3.11/dist-packa